In [1]:
"""
LWMの出力結果を用いて, 極値増大確率の算出とCDFの描画をする.
"""

import func
from csv import reader
import matplotlib.pyplot as plt
import numpy as np

MODEL = ['KF', 'YS']
INDEX = sorted([79*67 + 13, 79*64 + 26, 79*43 + 23, 79*63 + 43, 79*51 + 39, 79*42 + 60, 79*60 + 56, 79*34 + 40, 79*20 + 70, 79*20 + 55])
def calc_p(past_rv, past_pro, f_rv, f_pro):
    """
    確率密度関数から, 極値増大確率を算出する
    """
    P = 0
    for i in range(len(past_rv)):
        p = 0
        for j in range(len(f_rv)):
            if f_rv[j] > past_rv[i]:
                p += f_pro[j]
        P += past_pro[i] * p
    return P * 100

def GP(xi, sgm, y):
    """
    GPの1-CDF
    """
    if y <= 0:
        return 1
    else:
        return max(0, (1 + xi * y / sgm)) ** (-1/xi)


DATA_CNT = 50

# 描画用の画像を用意
fig = plt.figure(figsize=(18, 6))
for index in range(len(INDEX)):
    idx = INDEX[index]
    print("----------", 'point' + str(index+1), idx, "----------")
    for i in range(2):
        is_ok = True  # データ数がDATA_CNT以上あるかどうかのFlag(なければその地点は使用しない)
        for j in range(2):
            model = MODEL[i]
            # CSV名を指定するためにmodel名を変更
            if j == 0:  # 過去
                model = 'HPA_' + model
            else:  # 未来
                model = 'HFA_' + model + '_c0'
            with open('../pot_csv(100)(thr=6)/' + model + '_POT_DATA.csv', 'r') as csv_file:
                csv_reader = reader(csv_file)
                POT_ALL = list(csv_reader)
            POT = POT_ALL[idx] # 指定した場所のPOTを抽出
            # POTは文字列のリストになっているので, float型に変換する
            s = []
            for k in range(len(POT)):
                s.append(float(POT[k]))
            s = sorted(s, reverse=True)
            if len(POT) < 10:
                is_ok = False
                print("データ数：", len(POT), "データ数が不足しているため, この地点は使用できません")
            else:
                POT = s[:DATA_CNT]
                u = POT[-1]
                if j == 0:  # 過去
                    past_rv, past_pro, p_xi, p_sgm = func.lwm_gpd(data=POT, error=[0.005], thr=u, n=219143, n0=DATA_CNT, con=0.95)
                    u_p = u
                else:  # 将来
                    f_rv, f_pro, f_xi, f_sgm = func.lwm_gpd(data=POT, error=[0.005], thr=u, n=219143, n0=DATA_CNT, con=0.95)
                    u_f= u
        if is_ok:
            P = calc_p(past_rv, past_pro, f_rv, f_pro)
            print('{:.1f}'.format(P), '%の確率で極値増加')
            ax = fig.add_subplot(len(INDEX), 2, 2 * index + i+1)
            # MLのplot
            x = np.linspace(1, 50, 1000)
            ax.plot(x, [GP(xi=p_xi[1], sgm=p_sgm[1], y=x_ - u_p) for x_ in x], color='black', linestyle="dashdot", label='HPA_' + model[4:6])
            ax.plot(x, [GP(xi=p_xi[0], sgm=p_sgm[0], y=x_ - u_p) for x_ in x], color='red', linestyle="dashdot", label='HPA_' + model[4:6] + '(95%)')
            ax.plot(x, [GP(xi=p_xi[2], sgm=p_sgm[2], y=x_ - u_p) for x_ in x], color='red', linestyle="dashdot")
            ax.plot(x, [GP(xi=f_xi[1], sgm=f_sgm[1], y=x_ - u_f) for x_ in x], color='black', label=model[:-3])
            ax.plot(x, [GP(xi=f_xi[0], sgm=f_sgm[0], y=x_ - u_f) for x_ in x], color='red', label=model[:-3] + '(95%)')
            ax.plot(x, [GP(xi=f_xi[2], sgm=f_sgm[2], y=x_ - u_f) for x_ in x], color='red')
            ax.legend(fontsize=20)
            ax.set_xlim((0, 50))
            ax.set_ylim((0.005, 1))
            ax.set_xlabel("Hs[m]", fontsize=18, labelpad=1)
            ax.set_ylabel("1-CDF[per year]", fontsize=18, labelpad=1)
            ax.tick_params(axis='x', labelsize=18)
            ax.tick_params(axis='y', labelsize=18)
            ax.set_title(MODEL[i] + '(point' + str(index+1) + ')', fontsize=24)
            ax.set_yscale('log')
            ax.grid(which='major',color='black',linestyle='-', alpha=0.3)
            ax.grid(which='minor',color='black',linestyle='dotted', alpha=0.3)

# plt.subplots_adjust(hspace=0.3)
fig.subplots_adjust(left=0.07, right=0.98, bottom=0.1, top=0.92) #この1行を入れる
plt.savefig("../img/1-cdf_for_sum.png")
plt.show()

---------- point1 1635 ----------
16.297132742832154 22.339042948735433 91.483018743624
17.22951274567936 31.941531001274043 469.10467690546625
76.9 %の確率で極値増加
14.72620200259855 19.850460528771926 99.0217833261977
13.687072993256145 17.717630872345957 56.45774875406593
33.4 %の確率で極値増加
---------- point2 1650 ----------
19.131870542663652 24.546025036595907 86.90062721645877
17.43414449514146 22.411597620618117 100.02448985154794
42.9 %の確率で極値増加
15.57604830440743 17.00107375614604 28.936909612170222
15.884538752084929 19.091631526528296 44.683263640537746
77.2 %の確率で極値増加
---------- point3 2726 ----------
15.83442247511591 18.647715478393266 61.139626203155885
20.299416554549055 29.527514594447837 145.3707240328188
88.6 %の確率で極値増加
15.044347023408703 26.89452858844882 234.44272707624256
16.366318630795632 34.95131442764901 720.0986735494155
66.9 %の確率で極値増加
---------- point4 3378 ----------
19.528645448195075 31.015614825271086 320.5057532389896
17.18795851676573 24.383079530238156 137.3835223401